<a href="https://colab.research.google.com/github/danianamir/RL_UNITY/blob/main/RL_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# dependecies


In [ ]:
!git clone https://github.com/danianamir/RL_UNITY.git
%cd /content/RL_UNITY
!git pull origin main

In [ ]:
#package installs with pip_github......................................................................
%cd /content/RL_UNITY/ml-agents-envs
!pip install .
!pip install gymnasium
!pip install ray[rllib]

In [ ]:
# import.......................................................................................................
#mlagnets
import mlagents_envs
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.base_env import ActionTuple

#gym
import gymnasium as gym

from gymnasium import spaces

#ray[rllib]
import ray
from ray import air, tune
from ray.rllib.env.multi_agent_env import MultiAgentEnv

from ray.rllib.algorithms.ppo.ppo import PPOConfig
from ray.rllib.algorithms.ppo.ppo import PPO

from ray.rllib.algorithms.apex_ddpg import ApexDDPGConfig
from ray.rllib.algorithms.apex_ddpg import ApexDDPG


from ray.rllib.examples.policy.random_policy import RandomPolicy

from ray.rllib.algorithms.algorithm_config import AlgorithmConfig
from ray.tune.registry import register_env

from ray.rllib.evaluation.rollout_worker import RolloutWorker
from ray.rllib.evaluation.worker_set import WorkerSet
from ray.rllib.policy.policy import PolicySpec


#typing
from typing import Any, Dict, List, Optional, Tuple, Union
#nympy
import numpy as np
from pprint import pprint
import time
import random

import torch

In [ ]:
# give access to run the linux file ................................................................................
%cd
!chmod -R 755 /content/RL_UNITY/dn_server/dn_linux.x86_64
!chmod -R 755 /content/RL_UNITY/dn_server/UnityPlayer.so
!ls -l /content/RL_UNITY/dn_server/

%cd
!chmod -R 755 /content/RL_UNITY/3dball/dn_linux.x86_64
!chmod -R 755 /content/RL_UNITY/3dball/UnityPlayer.so
!ls -l //content/RL_UNITY/3dball/

# train gym

In [13]:
# create custom environmet for single agent setting
class CustomEnv(gym.Env):

  # first we initialize the type of action_space observation_space
  def __init__(self):
    #if you want to have 3 action and each of them have 5 diffrent value [0 1 2 3 4]
    #self.action_space = MultiDiscrete([5, 5, 5])
    self.action_space =spaces.Discrete(2)
    self.observation_space =spaces.Box(low=-100, high=100, shape=(1,))
    self.state = 0

  # we make the transition function that change the state for given action
  def step(self, action):

    if action == 0:
      self.state += 1
    elif action == 1:
      self.state -= 1


    if self.state==10 :
      reward =1
    else:
      reward=0

    done = (self.state >= 10 or self.state <= -10)
    info = {}
    truncated=False

    return np.array([self.state]), reward, done,truncated, info

  #reset the episod
  def reset(self, *, seed=None, options=None):
    self.state = 0
    info={}
    return np.array([self.state]) ,info






In [14]:
# register the env
tune.register_env("my",lambda _: CustomEnv())

In [16]:

# set config
config=(
       PPOConfig()
      .environment("my")
      .framework("tf")
      .rollouts(num_rollout_workers=1,
                rollout_fragment_length=200,
                )
      .training(
            lr=0.0003,
            lambda_=0.95,
            gamma=0.99,
            sgd_minibatch_size=256,
            train_batch_size=4000,
            num_sgd_iter=20,
            clip_param=0.2,
            model={"fcnet_hiddens": [256, 256]},
            _enable_learner_api=False
            )
      .rl_module(_enable_rl_module_api=False)
      .checkpointing(export_native_model_files=True)
      .resources()
)


#print the config
pprint(config.to_dict())

2023-09-03 14:12:09,300	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-09-03 14:12:09,306	WARNING algorithm_config.py:2572 -- Setting `exploration_config={'type': 'StochasticSampling'}` because you set `_enable_rl_module_api=False`. This exploration config was restored from a prior exploration config that was overriden when setting `_enable_rl_module_api=True`. This occurs because when RLModule API are enabled, exploration_config can not be set.


{'_AlgorithmConfig__prior_exploration_config': None,
 '_disable_action_flattening': False,
 '_disable_execution_plan_api': True,
 '_disable_initialize_loss_from_dummy_batch': False,
 '_disable_preprocessor_api': False,
 '_enable_learner_api': False,
 '_enable_rl_module_api': False,
 '_fake_gpus': False,
 '_is_atari': None,
 '_learner_class': None,
 '_tf_policy_handles_more_than_one_loss': False,
 'action_mask_key': 'action_mask',
 'action_space': None,
 'actions_in_input_normalized': False,
 'algorithm_config_overrides_per_module': {},
 'always_attach_evaluation_results': False,
 'auto_wrap_old_gym_envs': True,
 'batch_mode': 'truncate_episodes',
 'callbacks': <class 'ray.rllib.algorithms.callbacks.DefaultCallbacks'>,
 'checkpoint_trainable_policies_only': False,
 'clip_actions': False,
 'clip_param': 0.2,
 'clip_rewards': None,
 'compress_observations': False,
 'count_steps_by': 'env_steps',
 'create_env_on_driver': False,
 'custom_eval_function': None,
 'custom_resources_per_worker':

In [ ]:
#build algo
algo=config.build()


In [11]:


# train algo and print result
results=[]
for _ in range(6):
            result = algo.train()
            pprint(result)
            results.append(result)
            print("****************************************************************************************************************************************")
            if (
                result['timesteps_total'] >= 100000
                or result["episode_reward_mean"] >= 100000
                or result["episodes_total"]>=10000
            ):
                break



{'agent_timesteps_total': 8000,
 'config': {'_AlgorithmConfig__prior_exploration_config': None,
            '_disable_action_flattening': False,
            '_disable_execution_plan_api': True,
            '_disable_initialize_loss_from_dummy_batch': False,
            '_disable_preprocessor_api': False,
            '_enable_learner_api': False,
            '_enable_rl_module_api': False,
            '_fake_gpus': False,
            '_is_atari': None,
            '_learner_class': None,
            '_tf_policy_handles_more_than_one_loss': False,
            'action_mask_key': 'action_mask',
            'action_space': None,
            'actions_in_input_normalized': False,
            'algorithm_config_overrides_per_module': {},
            'always_attach_evaluation_results': False,
            'auto_wrap_old_gym_envs': True,
            'batch_mode': 'truncate_episodes',
            'callbacks': <class 'ray.rllib.algorithms.callbacks.DefaultCallbacks'>,
            'checkpoint_trainab

In [10]:
# save the whole algorithm

path_to_checkpoint = algo.save()
print(
    "An Algorithm checkpoint has been created inside directory:{} ".format(path_to_checkpoint)
)



#save the policy as a torch/tensorflow model
#it depends on framework configuration if "tf" export .pb  if "torch" export .pt
# insure the .checkpointing(export_native_model_files=True) is  set
policy=algo.get_policy()
policy.export_model("/content/policy_model")






#export as onnx format from tensorflow model .pp
policy.export_model("/content/onnx",onnx=15)



#export as onnx format from torch model .pt




An Algorithm checkpoint has been created inside directory:/root/ray_results/PPO_my_2023-09-03_14-00-47jd1jf14o/checkpoint_000001 
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



AttributeError: ignored

# train multi agnet

In [ ]:
class SimpleMultiAgentEnv(MultiAgentEnv):
    def __init__(self):
        self.num_agents = 2
        self._agent_ids = ["agent_0", "agent_1"]
        self.observation_space = spaces.Discrete(2)
        self.action_space = spaces.Discrete(2)
        self.initial_observations = {"agent_0": 0, "agent_1": 1}

    def reset(self, seed=None, options=None):
        info = {"agent_0": {}, "agent_1": {}}
        return self.initial_observations, info

    def step(self, action_dict):
        obs = {}
        rewards = {}
        dones = {}
        truncated = {}
        info = {}

        for key, value in action_dict.items():
            obs[key] = random.randint(0, 1)
            rewards[key] = 800
            if obs[key] >= 1000:
                dones[key] = True
            else:
                dones[key] = False
            truncated[key] = False
            info[key] = {}

        dones["__all__"] = False
        truncated["__all__"] = False

        return obs, rewards, dones, truncated, info


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:

tune.register_env("my",lambda _: SimpleMultiAgentEnv())

In [ ]:

config=PPOConfig()
config.environment("my")

policies = { "policy_1":PolicySpec(observation_space =spaces.Discrete(15),action_space=spaces.Discrete(15)),
            "policy_2" :PolicySpec(observation_space =spaces.Discrete(15),action_space=spaces.Discrete(15)),
             }
def policy_mapping_fn(agent_id, episode, worker, **kwargs):
  return "policy_1" if agent_id == 'agent_0' else ("policy_2" if agent_id == 'agent_1' else "policy_1")

config.multi_agent(policies=policies,policy_mapping_fn=policy_mapping_fn)
config.rollouts(num_rollout_workers=2)

2023-09-02 18:09:50,651	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.


In [ ]:
algo=config.build()

2023-09-02 18:09:52,887	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
/usr/local/lib/python3.10/dist-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/usr/local/lib/python3.10/dist-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by set

In [ ]:
print(algo.train())

{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'__all__': {'num_agent_steps_trained': 256.0, 'num_env_steps_trained': 4000.0, 'total_loss': 19.981413662306537}, 'policy_1': {'total_loss': 19.981413662306537, 'policy_loss': -0.011692280072156887, 'vf_loss': 10.0, 'vf_loss_unclipped': 5924266123.735608, 'vf_explained_var': -0.32959409086689, 'entropy': 2.69141259198504, 'mean_kl_loss': 0.015209753852401593, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.20000000298023224}, 'policy_2': {'total_loss': 9.990064018062437, 'policy_loss': -0.013269568848084095, 'vf_loss': 10.0, 'vf_loss_unclipped': 5924252253.61194, 'vf_explained_var': -0.2924032159197305, 'entropy': 2.6897527950404805, 'mean_kl_loss': 0.016668092678352207, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.20000000298023224}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 8000, 'num_agent_steps_trained': 0}, 'sampler_results': {'episode_re

# train unity


In [5]:
class Unity3DEnv(MultiAgentEnv):
    """A MultiAgentEnv representing a single Unity3D game instance.

    For an example on how to use this Env with a running Unity3D editor
    or with a compiled game, see:
    `rllib/examples/unity3d_env_local.py`
    For an example on how to use it inside a Unity game client, which
    connects to an RLlib Policy server, see:
    `rllib/examples/serving/unity3d_[client|server].py`

    Supports all Unity3D (MLAgents) examples, multi- or single-agent and
    gets converted automatically into an ExternalMultiAgentEnv, when used
    inside an RLlib PolicyClient for cloud/distributed training of Unity games.
    """

    # Default base port when connecting directly to the Editor
    _BASE_PORT_EDITOR = 5004
    # Default base port when connecting to a compiled environment
    _BASE_PORT_ENVIRONMENT = 5005
    # The worker_id for each environment instance
    _WORKER_ID = 0

    def __init__(
        self,
        file_name: str = None,
        port: Optional[int] = None,
        seed: int = 0,
        no_graphics: bool = False,
        timeout_wait: int = 300,
        episode_horizon: int = 1000,
    ):
        """Initializes a Unity3DEnv object.

        Args:
            file_name (Optional[str]): Name of the Unity game binary.
                If None, will assume a locally running Unity3D editor
                to be used, instead.
            port (Optional[int]): Port number to connect to Unity environment.
            seed: A random seed value to use for the Unity3D game.
            no_graphics: Whether to run the Unity3D simulator in
                no-graphics mode. Default: False.
            timeout_wait: Time (in seconds) to wait for connection from
                the Unity3D instance.
            episode_horizon: A hard horizon to abide to. After at most
                this many steps (per-agent episode `step()` calls), the
                Unity3D game is reset and will start again (finishing the
                multi-agent episode that the game represents).
                Note: The game itself may contain its own episode length
                limits, which are always obeyed (on top of this value here).
        """
        # Skip env checking as the nature of the agent IDs depends on the game
        # running in the connected Unity editor.
        self._skip_env_checking = True

        super().__init__()

        if file_name is None:
            print(
                "No game binary provided, will use a running Unity editor "
                "instead.\nMake sure you are pressing the Play (|>) button in "
                "your editor to start."
            )



        # Try connecting to the Unity3D game instance. If a port is blocked
        port_ = None
        while True:
            # Sleep for random time to allow for concurrent startup of many
            # environments (num_workers >> 1). Otherwise, would lead to port
            # conflicts sometimes.
            if port_ is not None:
                time.sleep(random.randint(1, 10))
            port_ = port or (
                self._BASE_PORT_ENVIRONMENT if file_name else self._BASE_PORT_EDITOR
            )
            # cache the worker_id and
            # increase it for the next environment
            worker_id_ = Unity3DEnv._WORKER_ID if file_name else 0
            Unity3DEnv._WORKER_ID += 1
            try:
                self.unity_env = UnityEnvironment(
                    file_name=file_name,
                    worker_id=worker_id_,
                    base_port=port_,
                    seed=seed,
                    no_graphics=no_graphics,
                    timeout_wait=timeout_wait,
                )
                print("Created UnityEnvironment for port {}".format(port_ + worker_id_))
            except mlagents_envs.exception.UnityWorkerInUseException:
                pass
            else:
                break

        # ML-Agents API version.
        self.api_version = self.unity_env.API_VERSION.split(".")
        self.api_version = [int(s) for s in self.api_version]

        # Reset entire env every this number of step calls.
        self.episode_horizon = episode_horizon
        # Keep track of how many times we have called `step` so far.
        self.episode_timesteps = 0

    def step(
        self, action_dict
    ):
        """Performs one multi-agent step through the game.

        Args:
            action_dict: Multi-agent action dict with:
                keys=agent identifier consisting of
                [MLagents behavior name, e.g. "Goalie?team=1"] + "_" +
                [Agent index, a unique MLAgent-assigned index per single agent]

        Returns:
            tuple:
                - obs: Multi-agent observation dict.
                    Only those observations for which to get new actions are
                    returned.
                - rewards: Rewards dict matching `obs`.
                - dones: Done dict with only an __all__ multi-agent entry in
                    it. __all__=True, if episode is done for all agents.
                - infos: An (empty) info dict.
        """

        # Set only the required actions (from the DecisionSteps) in Unity3D.
        all_agents = []
        for behavior_name in self.unity_env.behavior_specs:
            # New ML-Agents API: Set all agents actions at the same time
            # via an ActionTuple. Since API v1.4.0.
            if self.api_version[0] > 1 or (
                self.api_version[0] == 1 and self.api_version[1] >= 4
            ):
                actions = []
                for agent_id in self.unity_env.get_steps(behavior_name)[0].agent_id:
                    key = behavior_name + "_{}".format(agent_id)
                    all_agents.append(key)
                    actions.append(action_dict[key])
                if actions:
                    if actions[0].dtype == np.float32:
                        action_tuple = ActionTuple(continuous=np.array(actions))
                    else:
                        action_tuple = ActionTuple(discrete=np.array(actions))
                    self.unity_env.set_actions(behavior_name, action_tuple)
            # Old behavior: Do not use an ActionTuple and set each agent's
            # action individually.
            else:
                for agent_id in self.unity_env.get_steps(behavior_name)[
                    0
                ].agent_id_to_index.keys():
                    key = behavior_name + "_{}".format(agent_id)
                    all_agents.append(key)
                    self.unity_env.set_action_for_agent(
                        behavior_name, agent_id, action_dict[key]
                    )
        # Do the step.
        self.unity_env.step()

        obs, rewards, terminateds, truncateds, infos = self._get_step_results()

        # Global horizon reached? -> Return __all__ truncated=True, so user
        # can reset. Set all agents' individual `truncated` to True as well.
        self.episode_timesteps += 1
        if self.episode_timesteps > self.episode_horizon:
            return (
                obs,
                rewards,
                terminateds,
                dict({"__all__": True}, **{agent_id: True for agent_id in all_agents}),
                infos,
            )

        return obs, rewards, terminateds, truncateds, infos

    def reset(
        self, *, seed=None, options=None
    ):
        """Resets the entire Unity3D scene (a single multi-agent episode)."""
        self.episode_timesteps = 0
        self.unity_env.reset()
        obs, _, _, _, infos = self._get_step_results()
        return obs, infos

    def _get_step_results(self):
        """Collects those agents' obs/rewards that have to act in next `step`.

        Returns:
            Tuple:
                obs: Multi-agent observation dict.
                    Only those observations for which to get new actions are
                    returned.
                rewards: Rewards dict matching `obs`.
                dones: Done dict with only an __all__ multi-agent entry in it.
                    __all__=True, if episode is done for all agents.
                infos: An (empty) info dict.
        """
        obs = {}
        rewards = {}
        infos = {}
        for behavior_name in self.unity_env.behavior_specs:
            decision_steps, terminal_steps = self.unity_env.get_steps(behavior_name)
            # Important: Only update those sub-envs that are currently
            # available within _env_state.
            # Loop through all envs ("agents") and fill in, whatever
            # information we have.
            for agent_id, idx in decision_steps.agent_id_to_index.items():
                key = behavior_name + "_{}".format(agent_id)
                os = tuple(o[idx] for o in decision_steps.obs)
                os = os[0] if len(os) == 1 else os
                obs[key] = os
                rewards[key] = (
                    decision_steps.reward[idx] + decision_steps.group_reward[idx]
                )
            for agent_id, idx in terminal_steps.agent_id_to_index.items():
                key = behavior_name + "_{}".format(agent_id)
                # Only overwrite rewards (last reward in episode), b/c obs
                # here is the last obs (which doesn't matter anyways).
                # Unless key does not exist in obs.
                if key not in obs:
                    os = tuple(o[idx] for o in terminal_steps.obs)
                    obs[key] = os = os[0] if len(os) == 1 else os
                rewards[key] = (
                    terminal_steps.reward[idx] + terminal_steps.group_reward[idx]
                )

        # Only use dones if all agents are done, then we should do a reset.
        return obs, rewards, {"__all__": False}, {"__all__": False}, infos


In [6]:
# define funcction to make  myunity environemt
tune.register_env("unity3d",lambda _: Unity3DEnv(file_name="/content/RL_UNITY/dn_server/dn_linux.x86_64"))

In [7]:
policies = { "policy_1": PolicySpec(observation_space=spaces.Box(float("-inf"), float("inf"), (6,)),action_space=spaces.Box(float("-inf"), float("inf"), (2,))) }

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
  return "policy_1"

#set configs
config=(
    PPOConfig()
    .environment("unity3d")
    .multi_agent(policies=policies,policy_mapping_fn=policy_mapping_fn)
    .framework("torch")
    .rollouts(
            num_rollout_workers=1,
            rollout_fragment_length=200,
            )
    .training(
            lr=0.0003,
            lambda_=0.95,
            gamma=0.99,
            sgd_minibatch_size=256,
            train_batch_size=4000,
            num_sgd_iter=20,
            clip_param=0.2,
            model={"fcnet_hiddens": [512, 512]},
            )
    .resources(num_gpus=0)
    )


2023-09-03 07:24:28,766	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.


In [8]:
algo=config.build()

2023-09-03 07:24:31,470	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
/usr/local/lib/python3.10/dist-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/usr/local/lib/python3.10/dist-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by set

(RolloutWorker pid=10252) [UnityMemory] Configuration Parameters - Can be set up in boot.config
(RolloutWorker pid=10252)     "memorysetup-bucket-allocator-granularity=16"
(RolloutWorker pid=10252)     "memorysetup-bucket-allocator-bucket-count=8"
(RolloutWorker pid=10252)     "memorysetup-bucket-allocator-block-size=4194304"
(RolloutWorker pid=10252)     "memorysetup-bucket-allocator-block-count=1"
(RolloutWorker pid=10252)     "memorysetup-main-allocator-block-size=16777216"
(RolloutWorker pid=10252)     "memorysetup-thread-allocator-block-size=16777216"
(RolloutWorker pid=10252)     "memorysetup-gfx-main-allocator-block-size=16777216"
(RolloutWorker pid=10252)     "memorysetup-gfx-thread-allocator-block-size=16777216"
(RolloutWorker pid=10252)     "memorysetup-cache-allocator-block-size=4194304"
(RolloutWorker pid=10252)     "memorysetup-typetree-allocator-block-size=2097152"
(RolloutWorker pid=10252)     "memorysetup-profiler-bucket-allocator-granularity=16"
(RolloutWorker pid=1025

2023-09-03 07:24:43,854	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-09-03 07:24:43,869	WARNING deprecation.py:50 -- DeprecationWarning: `ValueNetworkMixin` has been deprecated. This will raise an error in the future!
2023-09-03 07:24:43,870	WARNING deprecation.py:50 -- DeprecationWarning: `LearningRateSchedule` has been deprecated. This will raise an error in the future!
2023-09-03 07:24:43,873	WARNING deprecation.py:50 -- DeprecationWarning: `EntropyCoeffSchedule` has been deprecated. This will raise an error in the future!
2023-09-03 07:24:43,875	WARNING deprecation.py:50 -- DeprecationWarning: `KLCoeffMixin` has bee

(RolloutWorker pid=10252) Created UnityEnvironment for port 5005


In [10]:
result=algo.train()
result

(RolloutWorker pid=10252) colide
(RolloutWorker pid=10252) UnityEngine.StackTraceUtility:ExtractStackTrace () (at /home/bokken/buildslave/unity/build/Runtime/Export/Scripting/StackTrace.cs:37)
(RolloutWorker pid=10252) UnityEngine.DebugLogHandler:LogFormat (UnityEngine.LogType,UnityEngine.Object,string,object[])
(RolloutWorker pid=10252) UnityEngine.Logger:Log (UnityEngine.LogType,object)
(RolloutWorker pid=10252) UnityEngine.Debug:Log (object)
(RolloutWorker pid=10252) MyAgent:OnCollisionEnter (UnityEngine.Collision) (at D:/DARS/code/unity/ml/Project/Assets/ML-Agents/Examples/dania/MyAgent.cs:42)
(RolloutWorker pid=10252) 
(RolloutWorker pid=10252) colide
(RolloutWorker pid=10252) UnityEngine.StackTraceUtility:ExtractStackTrace () (at /home/bokken/buildslave/unity/build/Runtime/Export/Scripting/StackTrace.cs:37)
(RolloutWorker pid=10252) UnityEngine.DebugLogHandler:LogFormat (UnityEngine.LogType,UnityEngine.Object,string,object[])
(RolloutWorker pid=10252) UnityEngine.Logger:Log (Unit

{'custom_metrics': {},
 'episode_media': {},
 'info': {'learner': {'__all__': {'num_agent_steps_trained': 256.0,
    'num_env_steps_trained': 4000.0,
    'total_loss': 0.06186205271435098},
   'policy_1': {'total_loss': 0.06186205271435098,
    'policy_loss': 0.002576873893626391,
    'vf_loss': 0.04846617842938144,
    'vf_loss_unclipped': 0.04846617842938144,
    'vf_explained_var': -0.10770912787403923,
    'entropy': 2.822500002650788,
    'mean_kl_loss': 0.054095001894230915,
    'curr_lr': 0.0003,
    'curr_entropy_coeff': 0.0,
    'curr_kl_coeff': 0.30000001192092896}},
  'num_env_steps_sampled': 4000,
  'num_env_steps_trained': 0,
  'num_agent_steps_sampled': 4000,
  'num_agent_steps_trained': 0},
 'sampler_results': {'episode_reward_max': 6.0,
  'episode_reward_min': 3.0,
  'episode_reward_mean': 4.0,
  'episode_len_mean': 1001.0,
  'episode_media': {},
  'episodes_this_iter': 3,
  'policy_reward_min': {'policy_1': 3.0},
  'policy_reward_max': {'policy_1': 6.0},
  'policy_rewa

# test

In [10]:
from ray.rllib.env.wrappers.unity3d_env import Unity3DEnv


tune.register_env(
        "unity3d",
        lambda c: Unity3DEnv(
            file_name=c["file_name"],
            no_graphics=False,
            episode_horizon=c["episode_horizon"],
        ),
    )


In [11]:

policies = { "policy_1": PolicySpec(observation_space=spaces.Box(float("-inf"), float("inf"), (8,)),action_space=spaces.Box(-1.0, 1.0, (2,), dtype=np.float32)) }

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
  return "policy_1"





config = (
        PPOConfig()
        .environment(
            "unity3d",
            env_config={
                "file_name":"/content/RL_UNITY/3dball/dn_linux.x86_64",
                "episode_horizon": 3000,
            },
        )
        .framework("torch")
        .rollouts(
            num_rollout_workers=1,
            rollout_fragment_length=200,
        )
        .training(
            lr=0.0003,
            lambda_=0.95,
            gamma=0.99,
            sgd_minibatch_size=256,
            train_batch_size=4000,
            num_sgd_iter=20,
            clip_param=0.2,
            model={"fcnet_hiddens": [512, 512]},
        )
        .multi_agent(policies=policies, policy_mapping_fn=policy_mapping_fn)
        .resources(num_gpus=0)
    )

2023-09-03 06:37:15,122	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.


In [8]:
algo=config.build()

2023-09-03 06:23:48,229	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
/usr/local/lib/python3.10/dist-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/usr/local/lib/python3.10/dist-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by set

(RolloutWorker pid=1802) [UnityMemory] Configuration Parameters - Can be set up in boot.config
(RolloutWorker pid=1802)     "memorysetup-bucket-allocator-granularity=16"
(RolloutWorker pid=1802)     "memorysetup-bucket-allocator-bucket-count=8"
(RolloutWorker pid=1802)     "memorysetup-bucket-allocator-block-size=4194304"
(RolloutWorker pid=1802)     "memorysetup-bucket-allocator-block-count=1"
(RolloutWorker pid=1802)     "memorysetup-main-allocator-block-size=16777216"
(RolloutWorker pid=1802)     "memorysetup-thread-allocator-block-size=16777216"
(RolloutWorker pid=1802)     "memorysetup-gfx-main-allocator-block-size=16777216"
(RolloutWorker pid=1802)     "memorysetup-gfx-thread-allocator-block-size=16777216"
(RolloutWorker pid=1802)     "memorysetup-cache-allocator-block-size=4194304"
(RolloutWorker pid=1802)     "memorysetup-typetree-allocator-block-size=2097152"
(RolloutWorker pid=1802)     "memorysetup-profiler-bucket-allocator-granularity=16"
(RolloutWorker pid=1802)     "memor

(RolloutWorker pid=1802) /usr/local/lib/python3.10/dist-packages/mlagents_envs/environment.py:94: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(RolloutWorker pid=1802)   unity_communicator_version = StrictVersion(unity_com_ver)
(RolloutWorker pid=1802) 2023-09-03 06:24:04,393	WARNING env.py:56 -- Skipping env checking for this experiment
(RolloutWorker pid=1802) 2023-09-03 06:24:04,401	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(RolloutWorker pid=1802) 2023-09-03 06:24:04,456	WARNING deprecation.py:50 -- DeprecationWarning: `ValueNetworkMixin` has been deprecated. This will r